In [1]:
import cv2
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
#Matplot Images
import matplotlib.image as mpimg
# Tensflor and Keras Layer and Model and Optimize and Loss
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
#PreTrained Model
from tensorflow.keras.applications import *
#Image Generator DataAugmentation
#Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
# Warnings Remove 
import warnings 
warnings.filterwarnings("ignore")
#Splitting Data 
# import splitfolders
from tensorflow.keras.preprocessing import image_dataset_from_directory
import random
from sklearn.cluster import KMeans
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

In [2]:
directory = "C:\\Users\\bvcha\\Desktop\\Senior Design Project\\Datasets\\PaddyDoctor\\Paddy_doctor\\train_images_paddy"

filepath =[]
label = []

folds = os.listdir(directory)

for fold in folds:
    f_path = os.path.join(directory , fold)
    
    imgs = os.listdir(f_path)
    
    for img in imgs:
        
        img_path = os.path.join(f_path , img)
        filepath.append(img_path)
        label.append(fold)
        
#Concat data paths with labels
file_path_series = pd.Series(filepath , name= 'filepath')
Label_path_series = pd.Series(label , name = 'label')
df_train = pd.concat([file_path_series ,Label_path_series ] , axis = 1)

In [3]:
from sklearn. model_selection import train_test_split
train_df , val_df = train_test_split(df_train ,train_size = 0.9 , shuffle = True ,random_state = 42 )

In [4]:
data_dir = 'C:\\Users\\bvcha\\Desktop\\Senior Design Project\\Datasets\\PaddyDoctor\\Paddy_doctor\\train_images_paddy'

print('Training Images:')
# creating the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.4,
    subset='training',
    seed=123,
    image_size=(256, 256),
    batch_size=128)

#Testing Augmented Data
print('Validation Images:')
validation_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, 
    validation_split=0.4,
    subset='validation',
    seed=123,
    image_size=(256, 256),
    batch_size=128)

Training Images:
Found 10407 files belonging to 10 classes.
Using 6245 files for training.
Validation Images:
Found 10407 files belonging to 10 classes.
Using 4162 files for validation.


In [5]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = 'C:\\Users\\bvcha\\Desktop\\Senior Design Project\\Datasets\\PaddyDoctor\\Paddy_doctor\\test_images_paddy',
    batch_size = 128,
    image_size = (256, 256),
    label_mode = None,
    shuffle=False
)
test_ds = validation_ds.map(lambda x, y: (x / 255.0, y))

Found 3469 files belonging to 1 classes.


In [6]:
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
# Val Data
validation_ds = validation_ds.map(lambda x, y: (x / 255.0, y))

In [7]:
DenseNet121_base = DenseNet121(weights='imagenet',
                                    include_top=False, 
                                   input_shape=(256, 256, 3),
                                     pooling = 'avg'
                                   )

# Freeze the pre-trained base model layers
DenseNet121_base.trainable = False



29084464/29084464 [==============================] - 5s 0us/step


In [8]:
model = Sequential()

# Add the pre-trained DenseNet121_base 
model.add(DenseNet121_base)

# Batch Normalization
model.add(BatchNormalization())

#Dropout 
model.add(Dropout(0.35))

# Add a dense layer with 120 units and ReLU activation function
model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))

model.add(Dense(220, activation='relu'))


# Add the output layer with 1 unit and sigmoid activation function for binary classification
model.add(Dense(10, activation='softmax'))

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1024)              7037504   
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 220)               225500    
                                                                 
 dense_1 (Dense)             (None, 220)               48620     
                                                                 
 dense_2 (Dense)             (None, 220)               48620     
                                                       

In [9]:
early_stopping = EarlyStopping(
    patience=10, 
    restore_best_weights=True,
)

# Train the model 
history = model.fit(train_ds,
                    epochs=10,
                    validation_data=validation_ds,
                    callbacks=early_stopping)

Epoch 1/10


49/49 [==============================] - 479s 10s/step - loss: 1.8764 - accuracy: 0.3321 - val_loss: 1.7903 - val_accuracy: 0.4397
Epoch 2/10
49/49 [==============================] - 475s 10s/step - loss: 1.3318 - accuracy: 0.5249 - val_loss: 1.4217 - val_accuracy: 0.5481
Epoch 3/10
49/49 [==============================] - 496s 10s/step - loss: 1.0624 - accuracy: 0.6444 - val_loss: 1.2477 - val_accuracy: 0.6081
Epoch 4/10
49/49 [==============================] - 466s 10s/step - loss: 0.8440 - accuracy: 0.7241 - val_loss: 0.9622 - val_accuracy: 0.7131
Epoch 5/10
49/49 [==============================] - 431s 9s/step - loss: 0.7314 - accuracy: 0.7681 - val_loss: 0.7454 - val_accuracy: 0.7734
Epoch 6/10
49/49 [==============================] - 411s 8s/step - loss: 0.6155 - accuracy: 0.8005 - val_loss: 0.6973 - val_accuracy: 0.7864
Epoch 7/10
49/49 [==============================] - 372s 8s/step - loss: 0.5366 - accuracy: 0.8269 - val_loss: 0.6126 - val_accuracy: 0.8073
Epoch 8